In [0]:
!pip install vecstack
from IPython.display import display
from sklearn import metrics
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from imblearn.over_sampling import SMOTE 
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from collections import Counter 

import warnings
warnings.filterwarnings("ignore")

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=44ddfb08d9c14f0e305702920d8c8f01c0fca9f92016cc7b3ba244aad7006901
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
trainfile =r'/gdrive/My Drive/CIS_508/Datasets/RevisedHomesiteTrain.csv'
train=pd.read_csv(trainfile)
train.head()

testfile=r'/gdrive/My Drive/CIS_508/Datasets/RevisedHomesiteTest.csv'
test=pd.read_csv(testfile)
test.head()

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,GeographicField64
0,13,22,4,4,4,4,3,3,3,4,13,22,13,23,1487.0,1.3045,16,0.9364,0.0006,-1,25,9,18,13,18,9,18,-1,19,10,13,8,17,2,10,-1,20,19,18,25,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,IL
1,4,5,8,14,8,14,7,12,8,13,13,22,13,23,564.0,1.1886,11,0.9919,0.0038,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,17,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,NJ
2,3,3,11,18,11,18,10,16,10,18,13,22,13,23,564.0,1.0670,15,0.8945,0.0038,-1,25,4,10,10,16,5,10,-1,11,11,18,5,13,2,13,-1,15,21,20,3,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,NJ
3,5,9,14,22,15,22,13,20,22,25,13,22,13,23,1113.0,1.2665,21,0.8870,0.0004,-1,25,24,24,23,23,24,24,25,25,23,22,24,25,2,7,-1,13,11,14,9,...,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,TX
4,12,21,4,5,4,5,4,4,4,5,13,22,13,23,935.0,1.0200,25,0.9153,0.0007,-1,25,2,3,3,5,2,5,-1,7,6,6,2,6,15,23,-1,17,4,8,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,CA


In [0]:
y_train=train.iloc[:,-1]
x_train=train.iloc[:,:-1]

x_test=test

colssale8_test=filter(lambda x: x!= 'GeographicField64',x_test.columns)
x_test=x_test[colssale8_test]

print(x_train.shape)
print(x_test.shape)
print(y_train.value_counts())

(65000, 595)
(173836, 595)
0    52738
1    12262
Name: QuoteConversion_Flag, dtype: int64


In [0]:
smt=SMOTE(ratio=0.6)

x_train,y_train=smt.fit_resample(x_train,y_train)

np.bincount(y_train)

array([52738, 31642])

In [0]:
#Lets perform STACKING- MODEL- 1

print('Ensembling gives the below results for MODEL-1:')


models_used = [GradientBoostingClassifier(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
s_train, s_test = stacking(models_used, x_train, y_train, x_test, regression=False,mode='oof_pred_bag', needs_proba=False,save_dir=None, metric=accuracy_score, n_folds=3,  stratified=True, shuffle=True,random_state=0, verbose=2)
                                


Ensembling gives the below results for MODEL-1:
task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.93920648]
    fold  1:  [0.93966437]
    fold  2:  [0.93561118]
    ----
    MEAN:     [0.93816068] + [0.00181243]
    FULL:     [0.93816070]

model  1:     [RandomForestClassifier]
    fold  0:  [0.91304039]
    fold  1:  [0.91833179]
    fold  2:  [0.91733627]
    ----
    MEAN:     [0.91623615] + [0.00229600]
    FULL:     [0.91623607]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.90770762]
    fold  1:  [0.90620778]
    fold  2:  [0.90549669]
    ----
    MEAN:     [0.90647070] + [0.00092156]
    FULL:     [0.90647073]



In [0]:
final_gb=GradientBoostingClassifier()

%time final_gb.fit(s_train,y_train)

train_predict=final_gb.predict(s_train)
print("accuracy Score for validation set:{}".format(final_gb.score(s_train,y_train)))
print("Confusion Matrix for Decision Tree")
print(classification_report(y_train,train_predict))
print(confusion_matrix(y_train,train_predict))

prediction=pd.DataFrame(final_gb.predict(s_test),columns=["QuoteConversion_Flag"])

pred_Probability =pd.DataFrame(final_gb.predict_proba(s_test))

pd.concat([test['QuoteNumber'],prediction['QuoteConversion_Flag']],axis=1).to_csv('/gdrive/My Drive/CIS_508/Assignments/Stacking_smote_homesite_modified.csv', index = None)

CPU times: user 2.43 s, sys: 23.4 ms, total: 2.45 s
Wall time: 2.46 s
accuracy Score for validation set:0.9447897353783921
Confusion Matrix for Decision Tree
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     52738
           1       0.97      0.91      0.94     42190

    accuracy                           0.94     94928
   macro avg       0.95      0.94      0.94     94928
weighted avg       0.95      0.94      0.94     94928

[[51408  1330]
 [ 3911 38279]]


In [0]:
smt=SMOTE(ratio=0.8)

x_train,y_train=smt.fit_resample(x_train,y_train)

np.bincount(y_train)

array([52738, 42190])

In [0]:
#Lets perform STACKING- MODEL- 2

print('Ensembling gives the below results for MODEL-2:')


models_used = [GradientBoostingClassifier(), RandomForestClassifier(), MLPClassifier() ]
      
s_train, s_test = stacking(models_used, x_train, y_train, x_test, regression=False,mode='oof_pred_bag', needs_proba=False,save_dir=None, metric=accuracy_score, n_folds=3,  stratified=True, shuffle=True,random_state=0, verbose=2)
                                


Ensembling gives the below results for MODEL-2:
task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.94381241]
    fold  1:  [0.94595790]
    fold  2:  [0.94459895]
    ----
    MEAN:     [0.94478976] + [0.00088622]
    FULL:     [0.94478974]

model  1:     [RandomForestClassifier]
    fold  0:  [0.92071167]
    fold  1:  [0.92029581]
    fold  2:  [0.92560521]
    ----
    MEAN:     [0.92220423] + [0.00241084]
    FULL:     [0.92220420]

model  2:     [MLPClassifier]
    fold  0:  [0.72165339]
    fold  1:  [0.87074142]
    fold  2:  [0.80936730]
    ----
    MEAN:     [0.80058737] + [0.06118074]
    FULL:     [0.80058571]



In [0]:
final_gb_model=RandomForestClassifier(n_estimators=80, max_depth=9, bootstrap=True)

%time final_gb_model.fit(s_train,y_train)

train_predict=final_gb_model.predict(s_train)
print("accuracy Score for validation set:{}".format(final_gb_model.score(s_train,y_train)))
print("Confusion Matrix for Decision Tree")
print(classification_report(y_train,train_predict))
print(confusion_matrix(y_train,train_predict))

prediction=pd.DataFrame(final_gb_model.predict(s_test),columns=["QuoteConversion_Flag"])

pred_Probability =pd.DataFrame(final_gb_model.predict_proba(s_test))

pd.concat([test['QuoteNumber'],prediction['QuoteConversion_Flag']],axis=1).to_csv('/gdrive/My Drive/CIS_508/Assignments/Stacking_smote_homesite_model2.csv', index = None)

CPU times: user 1.31 s, sys: 18.5 ms, total: 1.33 s
Wall time: 1.34 s
accuracy Score for validation set:0.9447897353783921
Confusion Matrix for Decision Tree
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     52738
           1       0.97      0.91      0.94     42190

    accuracy                           0.94     94928
   macro avg       0.95      0.94      0.94     94928
weighted avg       0.95      0.94      0.94     94928

[[51408  1330]
 [ 3911 38279]]


In [0]:
smt=SMOTE(ratio=1.0)

x_train,y_train=smt.fit_resample(x_train,y_train)

np.bincount(y_train)

array([52738, 52738])

In [0]:
#Lets perform STACKING- MODEL- 3

gb= GradientBoostingClassifier(n_estimators=120, max_depth=9, max_features='sqrt',warm_start=True,learning_rate=0.1)
rf=RandomForestClassifier(n_estimators=80, max_depth=13, bootstrap=True)
dt=DecisionTreeClassifier(max_leaf_nodes=400,max_depth=6,max_features=15,min_impurity_split=0.2)

models_used=[rf,gb,dt]

print('Ensembling gives the below results for MODEL-3:')


s_train, s_test = stacking(models_used, x_train, y_train, x_test, regression=False,mode='oof_pred_bag', needs_proba=False,save_dir=None, metric=accuracy_score, n_folds=4,  stratified=True, shuffle=True,random_state=0, verbose=2)
                                


Ensembling gives the below results for MODEL-3:
task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.93739097]
    fold  1:  [0.93795980]
    fold  2:  [0.93841019]
    fold  3:  [0.93765170]
    ----
    MEAN:     [0.93785317] + [0.00037943]
    FULL:     [0.93785316]

model  1:     [GradientBoostingClassifier]
    fold  0:  [0.95054987]
    fold  1:  [0.94899507]
    fold  2:  [0.95054612]
    fold  3:  [0.94978762]
    ----
    MEAN:     [0.94996967] + [0.00064263]
    FULL:     [0.94996966]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.82279105]
    fold  1:  [0.85157376]
    fold  2:  [0.83703732]
    fold  3:  [0.79968143]
    ----
    MEAN:     [0.82777089] + [0.01914589]
    FULL:     [0.82777125]



In [0]:
    final_mlp_model=MLPClassifier(hidden_layer_sizes=(21,),activation='relu',solver='sgd',learning_rate='adaptive',learning_rate_init=0.004,momentum=0.3,verbose=1, warm_start=True,beta_1=0.8)

  %time final_mlp_model.fit(s_train,y_train)

  train_predict=final_mlp_model.predict(s_train)
  print("accuracy Score for validation set:{}".format(final_mlp_model.score(s_train,y_train)))
  print("Confusion Matrix for Decision Tree")
  print(classification_report(y_train,train_predict))
  print(confusion_matrix(y_train,train_predict))

  prediction=pd.DataFrame(final_mlp_model.predict(s_test),columns=["QuoteConversion_Flag"])

  pred_Probability =pd.DataFrame(final_mlp_model.predict_proba(s_test))

  pd.concat([test['QuoteNumber'],prediction['QuoteConversion_Flag']],axis=1).to_csv('/gdrive/My Drive/CIS_508/Assignments/Stacking_smote_homesite_model3.csv', index = None)

Iteration 1, loss = 0.46687555
Iteration 2, loss = 0.28679876
Iteration 3, loss = 0.22592413
Iteration 4, loss = 0.20354809
Iteration 5, loss = 0.19386892
Iteration 6, loss = 0.18910952
Iteration 7, loss = 0.18656248
Iteration 8, loss = 0.18505335
Iteration 9, loss = 0.18405433
Iteration 10, loss = 0.18334613
Iteration 11, loss = 0.18279885
Iteration 12, loss = 0.18237236
Iteration 13, loss = 0.18201488
Iteration 14, loss = 0.18171697
Iteration 15, loss = 0.18145985
Iteration 16, loss = 0.18124083
Iteration 17, loss = 0.18105273
Iteration 18, loss = 0.18088622
Iteration 19, loss = 0.18074466
Iteration 20, loss = 0.18062003
Iteration 21, loss = 0.18051058
Iteration 22, loss = 0.18041581
Iteration 23, loss = 0.18033337
Iteration 24, loss = 0.18026205
Iteration 25, loss = 0.18019632
Iteration 26, loss = 0.18013827
Iteration 27, loss = 0.18008766
Iteration 28, loss = 0.18004643
Iteration 29, loss = 0.18000515
Iteration 30, loss = 0.17997038
Iteration 31, loss = 0.17993769
Iteration 32, los